In [ ]:
USE_PRIVATE_DISTRO = True
DRIVE_BASE_DIR = '/content/drive/MyDrive/SMC 10/DDSP-10/' 

DRIVE_DISTRO = DRIVE_BASE_DIR + 'dist/ddsp-1.2.0.tar.gz'

if USE_PRIVATE_DISTRO:
    print("[INFO] Using private distro. Be careful.")
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install -qU "$DRIVE_DISTRO"[data_preparation]
else:
    !pip install -qU ddsp[data_preparation]==1.2.0

%tensorflow_version 2.x
import tensorflow as tf

from ddsp.colab.colab_utils import play, specplot
from ddsp.synths import AmplitudeModulation
from ddsp.training import data

import glob
import numpy as np
import os
import soundfile as sf
import warnings

import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

In [ ]:
sns.set(style="whitegrid")
warnings.filterwarnings("ignore")

# AM Synth

In [ ]:
OUTPUT_FOLDER = 'noadsr' #@param {type: "string"}

DRIVE_AUDIO_DIR = DRIVE_BASE_DIR + 'audio/' + OUTPUT_FOLDER + '/'
COLAB_AUDIO_DIR = 'data/audio/' 
AUDIO_FILEPATTERN = COLAB_AUDIO_DIR + '/*'

!mkdir -p "$DRIVE_AUDIO_DIR"
!mkdir -p "$COLAB_AUDIO_DIR"

In [ ]:
FRAME_RATE = 250
DURATION = 4

n_frames = FRAME_RATE * DURATION


#Carrier amp
# carr_amp = np.concatenate((np.linspace(-10,1,25), 
#                            np.ones(575), 
#                            np.linspace(1,-10,250), 
#                            -10*np.ones(150)), axis=0)
carr_amp = 3*np.ones(n_frames)
carr_amp = carr_amp[np.newaxis, :, np.newaxis]


#Modulator amp 
mod_amp = 3*np.ones(n_frames)
mod_amp = mod_amp[np.newaxis, :, np.newaxis]


for SAMPLE_RATE in [16000]:
# for SAMPLE_RATE in [16000, 48000]:

  n_samples = SAMPLE_RATE * DURATION
  am_synth = AmplitudeModulation(n_samples=n_samples,
                                 sample_rate=SAMPLE_RATE)
  for freqs in [5, 125, 250]:
  # for freqs in [5, 25, 125]:

    INSTRUMENT = 'sr{}k_mf{}'.format(SAMPLE_RATE//1000, freqs)
    DRIVE_AUDIOFILE = DRIVE_AUDIO_DIR + INSTRUMENT + '.wav'
    COLAB_AUDIOFILE = COLAB_AUDIO_DIR + INSTRUMENT + '.wav'

    print('1. Render ' + INSTRUMENT + '.wav')

    #Mod freq 
    mod_freq = freqs * np.ones(n_frames)
    mod_freq = mod_freq[np.newaxis, :, np.newaxis]

    output = []

    for f in range(0,24):

      #Main freq
      carr_freq = 220 * np.power(2, f/12) * np.ones(n_frames)
      carr_freq = carr_freq[np.newaxis, :, np.newaxis]

      audio = am_synth(carr_amp, carr_freq, mod_amp, mod_freq)
      output = np.concatenate((output, audio[0]))

    # display(Audio(output, rate=sample_rate, autoplay=False))
    output /= np.max(np.abs(output),axis=0)*1.5
    sf.write(DRIVE_AUDIOFILE, output, SAMPLE_RATE)

    DRIVE_DATASET_DIR = DRIVE_AUDIO_DIR + INSTRUMENT + '_fr{}_dataset'.format(FRAME_RATE) 
    COLAB_RECORD = 'data/' + INSTRUMENT + '_fr{}_dataset/train.synthrecord'.format(FRAME_RATE) 
    RECORD_FILEPATTERN = COLAB_RECORD + '*'

    !mkdir "$DRIVE_DATASET_DIR"

    print('2. Copy {} to {}'.format(DRIVE_AUDIOFILE, COLAB_AUDIOFILE))
    !cp "$DRIVE_AUDIOFILE" "$COLAB_AUDIOFILE"

    print('3. Create ' + INSTRUMENT + '_fr{} dataset'.format(FRAME_RATE))
    if not glob.glob(COLAB_AUDIOFILE):
      raise ValueError('No audio files found')

    !ddsp_dataset_from_notes \
    --input_audio_filepatterns=$COLAB_AUDIOFILE \
    --output_tfrecord_path=$COLAB_RECORD \
    --sample_rate=$SAMPLE_RATE \
    --frame_rate=$FRAME_RATE \
    --example_secs=4 \
    --midi_notes='57','58','59','60','61','62','63','64','65','66','67','68','69','70','71','72','73','74','75','76','77','78','79','80' \
    --alsologtostderr

    print('4. Save to {}'.format(DRIVE_DATASET_DIR)) 
    !cp $RECORD_FILEPATTERN "$DRIVE_DATASET_DIR"

    print('\n\n\n')